# 1. 排序层的基本概念
### 1.1 排序层的作用
- 精细化排序：对召回结果进行精准打分和排序
- 多目标优化：平衡点击率、转化率、时长等多个目标
- 个性化展现：根据用户实时上下文调整排序

### 1.2 排序流程

- `召回候选集 → 特征工程 → 排序模型 → 重排序 → 最终结果`

---
# 2.排序层的目标

- 排序层的主要目标：
    - 对用户和物品的匹配程度做 精准打分。
    - 学习一个 排序函数 𝑓(𝑢,𝑖)输入是用户特征 `𝑢` 和物品特征` 𝑖`，输出是一个分数` 𝑠`。
    - 分数可以理解为：用户对该物品的兴趣概率（比如点击率 `CTR`、转化率 `CVR`）。
    - 最终按照分数从高到低排序，推荐给用户。

---
# 3.排序层常见输入特征

排序层特征设计通常非常丰富，常分为以下几类：

- 3.1.用户特征
    - 静态特征：性别、年龄、地域、会员等级
    - 行为特征：近 7 天点击数、最近一次活跃时间、历史偏好分布
- 3.2.物品特征
    - 静态特征：类别、价格、发布时间、内容 embedding
    - 热度特征：点击率、曝光量、最近一小时热度
- 3.3.交叉特征（用户 × 物品）
    - 用户与物品类别的匹配度
    - 用户历史平均停留时长 × 当前物品视频时长
    - `embedding` 内积（衡量相似度）
    - `embedding` 理解为一种“数学化的表示”，它的核心思想是：将高维、稀疏的离散数据（如文字、ID号），映射到一个低维、稠密的连续向量空间中去。
- 3.4.上下文特征
    - 时间（早晨/晚上）
    - 地理位置
    - 设备类型（iOS/Android/Web）
---

# 4 特征工程 特征类型
### 4.1用户特征

In [ ]:
class UserFeatures:
    def __init__(self):
        self.static_features = {}  # 静态特征
        self.dynamic_features = {}  # 动态特征
        self.behavior_features = {}  # 行为特征
    
    def extract_static_features(self, user_profile):
        """提取用户静态特征"""
        return {
            'user_id': user_profile['user_id'],
            'age': user_profile['age'],
            'gender': user_profile['gender'],
            'location': user_profile['location'],
            'register_days': (datetime.now() - user_profile['register_time']).days
        }
    
    def extract_dynamic_features(self, user_behavior):
        """提取用户动态特征"""
        return {
            'recent_click_count': len(user_behavior.get('clicks', [])),
            'recent_purchase_count': len(user_behavior.get('purchases', [])),
            'active_level': self._calculate_active_level(user_behavior),
            'preference_categories': self._extract_preference_categories(user_behavior)
        }
    
    def extract_behavior_features(self, user_history):
        """提取用户行为序列特征"""
        return {
            'click_sequence': user_history['clicks'][-50:],  # 最近50次点击
            'dwell_time_avg': np.mean([item['dwell_time'] for item in user_history['clicks']]),
            'preference_vector': self._calculate_preference_vector(user_history)
        }

### 4.2物品特征

In [ ]:
class ItemFeatures:
    def __init__(self):
        self.content_features = {}
        self.statistical_features = {}
    
    def extract_content_features(self, item_info):
        """提取物品内容特征"""
        return {
            'item_id': item_info['item_id'],
            'category': item_info['category'],
            'price': item_info['price'],
            'brand': item_info['brand'],
            'tags': item_info['tags'],
            'content_embedding': item_info.get('content_embedding', [])
        }
    
    def extract_statistical_features(self, item_stats):
        """提取物品统计特征"""
        return {
            'historical_ctr': item_stats['clicks'] / (item_stats['impressions'] + 1e-6),
            'purchase_rate': item_stats['purchases'] / (item_stats['clicks'] + 1e-6),
            'avg_rating': item_stats.get('avg_rating', 0),
            'review_count': item_stats.get('review_count', 0),
            'popularity_score': self._calculate_popularity_score(item_stats)
        }

### 4.3上下文特征

In [ ]:
class ContextFeatures:
    def extract_context_features(self, context_info):
        """提取上下文特征"""
        current_time = context_info['timestamp']
        return {
            'hour_of_day': current_time.hour, # 提取 小时 时间戳  时刻数据 
            'day_of_week': current_time.weekday(),  # 提取 周 时间戳
            'is_weekend': 1 if current_time.weekday() >= 5 else 0,  # 
            'season': self._get_season(current_time.month),  
            'device_type': context_info['device_type'],
            'network_type': context_info['network_type'],
            'location_context': context_info.get('location', 'unknown')
        }

### 4.4交叉特征

In [ ]:
class CrossFeatures:
    def extract_cross_features(self, user_features, item_features, context_features):
        """提取交叉特征"""
        return {
            'user_item_category_match': 1 if item_features['category'] in user_features['preference_categories'] else 0,
            'price_user_affordability': self._calculate_affordability(user_features, item_features),
            'time_user_preference': self._calculate_time_preference(user_features, context_features),
            'user_item_similarity': cosine_similarity(
                [user_features['preference_vector']],
                [item_features['content_embedding']]
            )[0][0]
        }

# 5.排序层常见模型

排序层的核心是 点击率预估（CTR prediction） 或 转化率预估（CVR prediction），常见模型：

- 5.1.线性模型（早期）:
    - `LR` (Logistic Regression)：易解释，速度快，但不能捕捉非线性特征。

- 5.2.树模型:
    - `GBDT` / `XGBoost` / `LightGBM`：在小规模特征场景下表现优秀。
    
    - 但在大规模稀疏特征（比如 embedding）中表现有限。

- 5.3.深度学习模型（主流）

    - `Wide` & `Deep`：结合记忆能力（Wide）和泛化能力（Deep）。
    
    - `DeepFM`：在 `Wide` & `Deep` 的基础上，用` FM `自动学习特征交叉。
    
    - `DIN` (Deep Interest Network)：引入注意力机制，建模用户对候选` item `的兴趣相关性。
    
    - `DIEN`：在` DIN `的基础上加入` GRU`，建模用户兴趣随时间的演化。
    
    - `DCN` / `xDeepFM`：更高阶的特征交叉。

- 5.4.多任务学习模型

    - ESMM (Entire Space Multi-Task Model)：同时建模 CTR 和 CVR，避免样本稀疏问题。
    
    - MMoE (Multi-gate Mixture-of-Experts)：同时优化多个目标（点击、点赞、分享）。


---
# 6.排序层的优化目标

排序层的优化目标依赖于业务：

6.1、单目标优化：

- 预测`点击率`（`CTR`） → 最大化点击

- 预测`转化率`（`CVR`） → 最大化购买

6.2、多目标优化：

- 预测点击率、转化率、完播率等多个指标 → 综合加权排序

6.3、长期价值优化

- 预测用户生命周期价值（`LTV`）

- 不仅优化眼前点击，还优化长期留存

---
# 7.排序层的训练流程

7.1、构建训练样本：

- 正样本：用户点击/购买的` item`

- 负样本：用户看到但没点击的` item`

7.2、特征工程：

- 类别特征 → `One-hot` / `Embedding`

- 数值特征 → `标准化` / `分桶`

7.3、模型训练：

- 损失函数一般是 `交叉熵`（预测点击概率）。

7.4、线上预测：

- 给候选` item `打分，按分数排序。

---
# 8、排序层的挑战

- 8.1.点击率陷阱：高点击但低转化的内容（比如标题党）。

解决：多任务建模，加入转化/留存目标。

- 8.2.冷启动问题：新用户、新物品缺少历史行为。

解决：利用内容特征 / 迁移学习。

- 8.3.计算效率问题：实时计算几百上千候选的打分，延迟要 < `100ms`。

解决：模型蒸馏、轻量化模型。

## ✅ 总结：
排序层是推荐系统的 `精细打分环节`，目标是预测用户对候选` item `的兴趣概率。
它依赖 丰富的特征工程 + 强大的模型（深度学习为主），并且需要兼顾业务目标（点击、转化、留存）。